In [1]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY") 

# check the sever type according to API key
if openai.api_key.startswith("sk-"): # OpenAI
    server_type = "OpenAI"
else:
    server_type = "AzureOpenAI"

# update env info if serer type is AzureOpenAI
if server_type == "AzureOpenAI": # AzureOpenAI
    # Load config values
    with open(r'azure_config.json') as config_file:
        config_details = json.load(config_file)
        # Set AzureOpenAI API values
        openai.api_type = config_details["OPENAI_API_TYPE"]
        os.environ['OPENAI_API_BASE'] = config_details["OPENAI_API_BASE"]
        openai.api_base=config_details["OPENAI_API_BASE"]
        os.environ["OPENAI_API_VERSION"] = config_details["OPENAI_API_VERSION"]
        openai.api_version=config_details["OPENAI_API_VERSION"]

# check the enironment variable
print(f"sever_type: {server_type} | api_type: {openai.api_type}")

sever_type: AzureOpenAI | api_type: azure


In [2]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

print(prompt)


Summarize the text delimited by triple backticks \ 
into a single sentence.
```
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
```



In [3]:
messages = [{"role": "user", "content": prompt}]
response = openai.ChatCompletion.create(
    deployment_id='gpt-35-turbo',
    # model='gpt-3.5-turbo-16k',
    messages=messages,
    temperature=0, # this is the degree of randomness of the model's output
)

print(response)

{
  "id": "chatcmpl-7UXEVDaooxbMk5VJejxmFz782c6XA",
  "object": "chat.completion",
  "created": 1687511943,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs."
      }
    }
  ],
  "usage": {
    "completion_tokens": 37,
    "prompt_tokens": 135,
    "total_tokens": 172
  }
}


In [4]:
response.keys()

dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])

In [5]:
print(response['choices'])

[<OpenAIObject at 0x10d52be90> JSON: {
  "index": 0,
  "finish_reason": "stop",
  "message": {
    "role": "assistant",
    "content": "Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs."
  }
}]


In [6]:
response.choices[0].message["content"]

'Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.'

In [7]:
response['usage'], response['usage']['total_tokens']

(<OpenAIObject at 0x10d52bf50> JSON: {
   "completion_tokens": 37,
   "prompt_tokens": 135,
   "total_tokens": 172
 },
 172)

## Helper Function

In [8]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        deployment_id='gpt-35-turbo',
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [9]:
response = get_completion(prompt)
print(response)

To guide a model towards the desired output and reduce irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.


In [10]:
prompt = f"""what is the answer to life, the universe, and everything?"""

def get_completion(prompt):
    messages=[
            {"role": "system", "content": "You are an AI assistant that helps people find information."},
            {"role": "user", "content": prompt}
        ]

    response = openai.ChatCompletion.create(
        deployment_id='gpt-35-turbo',
        messages = messages,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    return response.choices[0].message["content"]

response = get_completion(prompt)
print(response)

According to Douglas Adams' book "The Hitchhiker's Guide to the Galaxy," the answer to the ultimate question of life, the universe, and everything is 42.


Top P: 

Similar to temperature, this controls randomness but uses a different method. Lowering Top P will narrow the model’s token selection to likelier tokens. Increasing Top P will let the model choose from tokens with both high and low likelihood. Try adjusting temperature or Top P but not both.